In [58]:
import pandas as pd
import numpy
import pyodbc
from vega_datasets import data
import sqlalchemy
from sqlalchemy import types
import time

In [2]:
def ConnctDB(driver, server, port, database, username, password):
    conn = pyodbc.connect(
        f"DRIVER={driver};SERVER={server},{port};DATABASE={database};UID={username};PWD={password};MULTI_HOST=1"
        )
    return conn

In [63]:
#### DB 參數設置
driver = "MySQL ODBC 8.0 ANSI Driver"
server = 'localhost'
port = '3306'
database = 'Project_Test'
username = 'root'
password = 'your_password'

In [60]:
#### how to get driver name
#### choose the name fit your DB
print(pyodbc.drivers())

['SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)', 'MySQL ODBC 8.0 ANSI Driver', 'MySQL ODBC 8.0 Unicode Driver']


In [64]:
#### get experiment dataset
df = data.iris()
print(df.head())

#### table name and column name for DB and connect DB
table_name = 'iris_{}'
col_str = ['sepalLength', 'sepalWidth', 'petalLength', 'petalWidth', 'species']
conn = ConnctDB(driver, server, port, database, username, password)

   sepalLength  sepalWidth  petalLength  petalWidth species
0          5.1         3.5          1.4         0.2  setosa
1          4.9         3.0          1.4         0.2  setosa
2          4.7         3.2          1.3         0.2  setosa
3          4.6         3.1          1.5         0.2  setosa
4          5.0         3.6          1.4         0.2  setosa


In [67]:
#### basic way to store data to DB
#### exmaple 1
df.to_sql(con=conn, name=table_name.format('to_sql'),
          if_exists='replace',
          dtype={'sepalLength': types.NUMERIC(),
                 'sepalWidth': types.NUMERIC(), 
                 'petalLength': types.NUMERIC(), 
                 'petalWidth': types.NUMERIC(), 
                 'species': types.VARCHAR(length=50)})

#### example 2
#### turn data type from python dtype to sql type
#### example: https://stackoverflow.com/questions/4165143/easy-convert-betwen-sqlalchemy-column-types-and-python-data-types/24550149#24550149
#### sqlalchemy-types: https://code-maven.com/slides/python/sqlalchemy-types
type_py2sql_dict = {str: types.VARCHAR(50),
                    numpy.float64: types.Float(precision=3, asdecimal=True)}

py_type = [type(i) for i in df.loc[0]]
sql_type = [type_py2sql_dict[i] for i in py_type]
col_type_dict = dict(zip(df.columns, sql_type))
df.to_sql(con=conn, name=table_name.format('to_sql'),
          if_exists='replace',
          dtype=col_type_dict)

In [68]:
#### when dataset is large, use fast_executemany=True and executemany
#### insert needed data into destination DB
s = time.time()
sql = f"INSERT INTO {table_name} ({','.join(col_str)}) VALUES ({','.join(['?']*len(col_str))})"
params = df.values.tolist()

cursor = conn.cursor() 
cursor.columns(table=table_name.format('executemany'))
cursor.fast_executemany = True
cursor.executemany(sql, params)

print('executemany:',numpy.round((time.time()-s)/60, 3), 'mins')

executemany: 0.0 mins
